Logan Williamson
===
Homework 11
====
####CHE 116: Numerical Methods and Statistics
Prof. Andrew White

Version 1.4 (4/2/2015)

----

In [1]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, pi, erf
import seaborn
#seaborn.set_style("white")
import scipy.stats
import numdifftools as nd

0. Warm-up (3 EC Points)
===

Correct three problems you've gotten wrong on the last two homeworks. State the problem, what your mistake was, and what lecture you could go to learn more about it.

####HW 7 #5

I messed up my function for T(x) causing my concentration of reactants to be incorrect. The correct function is shown below. This material was covered in Unit 7 Lecture 2.

####HW 7 #6

My constaint was invalid. The correct constaint is shown below. Constainted optimization was covered in Unit 7 Lecture 2.

####HW 7 #8

I failed to state what p(t) was. This is shown below. Working with functions of random variables was covered in Unit 8 Lecture 1.

$p(t) = \lambda e^{-\lambda t}$

***

1. Conceptual Questions (4 Points)
===

1. The right-hand side of a likelihood is the probability density function evaluated at a point if we're working with a continuous distribution. Why is evaluating the probability density function at a point a likelihood and not the same as a probability?

2. If we have two observations, our likelihood will be $p(x_1 | \theta) p(x_2 | \theta)$. What assumption is implicit about the observations?

3. Is bootstrap resampling a non-parametric or parametric method?

4. What about maximum likelihood estimation?


####1 Solutions

1. You cannot have point probabilily, you must have an interval, which requires integration for a continuous distribution. Therefore when you evaluate the probabilty distribution at a point it is a likelihood rather than a probability.
2. The samples are independent.
3. Bootstrap is a non-parametric resampling method.
4. MLE is parametric method because you are assuming that the data comes from a chosen probability distribution. 

***

2. Exercises (12 Points)
===

1. If I have [4.3, 4.2, 4.5, 3.9, 4.0, 4.3, 3.9, 3.8, 3.6, 4.1], what is the 95% confidence interval for the mean? Use a $t$-distribution

1. Repeat problem 1 using bootstrap resampling instead of a $t$-distributions. Use the fact that $\hat{\mu}$, the MLE for mean, is just the sample mean of the data (do not do any maximization)

2. If I have a sample of $x = 2$, what is the likelihood for a $\lambda = 2$ exponential distribution? 

3. What is the log-likelihood function for an exponential distribution? Do not use the values from problem 2, just give the function.

4. My model equation from last unit for a regression was $y = \beta x + \alpha + \epsilon$. What term is random and what is deterministic?

5. I have two models. One is a proposed and another is a restricted version of the proposed model. The proposed model has 2 more dimensions than the restricted model. If $\mathcal{l}_0 = 45$ is the likelihood of the first model and $\mathcal{l}_1 = 56$ is the likelihood of the proposed model, should I use the proposed model? *Hint: This is a likelihood ratio test.*

6. You have two models being compared: a null model and proposed model. Can you get a higher likelihood for the null model (assume $q \geq 1$)? 

####2.1 Solution

In [2]:
numbers = np.array([4.3, 4.2, 4.5, 3.9, 4.0, 4.3, 3.9, 3.8, 3.6, 4.1])
T = scipy.stats.t.ppf(0.975, df = len(numbers))
x_bar = np.mean(numbers)
sigma_x = np.sqrt(np.var(numbers, ddof = 1))
S = sigma_x / np.sqrt(len(numbers))
y = S*T
print 'With a confidence level of 5%, the mean is',x_bar, '+-', y

With a confidence level of 5%, the mean is 4.06 +- 0.191383739637


####2.2 Solution

In [3]:
N = 10000
boot_results = np.empty((N))
for i in range(N):
    sampled_numbers = np.random.choice(numbers, size=len(numbers), replace=True)
    boot_results[i] = np.mean(sampled_numbers)
    
y = np.percentile(boot_results, [2.5, 97.5])
print '95% Confidence interval for mean using quantiling:',y

95% Confidence interval for mean using quantiling: [ 3.9   4.22]


####2.3 Solution

In [4]:
def func(x, lamb):
    return lamb * np.exp(-lamb*x)
print 'the likelihood is', func(2,2)

the likelihood is 0.0366312777775


####2.4 Solution
$$\mathcal{l}(\lambda; X) = N\ln (\lambda) - \lambda\sum_i^N x_{i} $$

####2.5 Solution

 * The random term is the noise (epsilon).
 * The deterministic terms are x, beta, and alpha.

####2.6 Solution

In [5]:
ll_0 = 45 #Null Model
ll_1 = 56 #Proposed Model
#Compute the LR sample statistic
LR = 2 * (ll_1 - ll_0)

print ll_0, ll_1, LR

#Now do hypothesis test using chi^2
p = 1 - scipy.stats.chi2.cdf(LR, 2)
print 'p-value =',p,'therefore we reject the null model and use the proposed model.'

45 56 22
p-value = 1.67017007903e-05 therefore we reject the null model and use the proposed model.


####2.7 Solution

No, you cannot get a higher likelihood for the null model. This is because the proposed model has more dimensons, whose natural logs are summed together, which are always postive, causing the likelihood of the proposed model to always be greater than the null model.

***

3. Exponential Distribution (5 Points)
====

Provide a 95% confidence interval for the rate of an exponential distribution given the following data. *Hint: The best-fit is about 0.28*

In [6]:
prob_3_data = np.array([3.2244, 4.1327,  3.0415,  3.3252,  3.0334,  3.3537,  3.0778,  3.1752,  3.3267,  4.0439,  3.3105,  4.5029,  4.3982,  4.3311,  3.1275])

####3 Solution

In [7]:
def exp_ll(lamb, data):
    return len(data) * np.log(lamb) - lamb * np.sum(data)
    
N = 10000 #The number of times we'll estimate the parameters
bootstrap_results = np.empty((N)) #Create a place to store the data
for i in range(N):
    sampled_data = np.random.choice(prob_3_data, size=len(prob_3_data), replace=True)
    result = scipy.optimize.minimize(lambda x: -exp_ll(x, sampled_data), x0=[1.], bounds = [(0.001, 100)])
    bootstrap_results[i] = result.x
print '95% Confidence interval for rate of exponential using quantiling:', np.percentile(bootstrap_results, [2.5, 97.5])


95% Confidence interval for rate of exponential using quantiling: [ 0.26017092  0.30224608]


***

4. The Old Toad (7 Points)
===

Recall the old-toad distribution. You have two die: a fair and unfair die. You have a probability, called $p$, for when you choose to use the unfair die. If you're using the fair die, the probability of rolling a number is $1/6$. If you're rolling the unfair die, the probability of rolling a number is $x / 21$, where $x$ is the number. Compute a 95% confidence interval for $p$ using the data below. *Hint: $P(x | \theta)$ is just what we normally write as $P(x)$, the marginal. Recall the special rules for finding a marginal distribution when there are conditionals in our distribution. The best-fit is a little over 0.5*

In [8]:
prob_4_data = np.array([1, 5,  5,  6,  3,  6,  6,  4,  4,  6,  6,  6,  3,  2,  3,  2,  2,  5,  2,  5,  2,  3,  3,  5,  1,  2,  4,  4,  6,  1,  2,  6,  2,  6,  3,  5,  6,  5,  3,  2,  5,  6,  6,  3,  6,  5,  1,  4,  4,  3,  4,  2,  5,  6,  4,  1,  3,  6,  3,  3,  6,  4,  6,  3,  3,  2,  2,  6,  6,  6,  6,  3,  4,  6,  5,  3,  6,  5,  3,  6,  4,  1,  6,  5,  2,  3,  4,  3,  4,  2,  6,  2,  1,  6,  3,  5,  1,  3,  6,  4])

####4 Solution

In [9]:
def distribution_ll(p, data):
    return np.sum(np.log((data / 21.) * p + (1/6.)*(1-p)))
    
N = 10000 #The number of times we'll estimate the parameters
bootstrap_results = np.empty((N)) #Create a place to store the data
for i in range(N):
    sampled_data = np.random.choice(prob_4_data, size=len(prob_4_data), replace=True)
    result = scipy.optimize.minimize(lambda x: -distribution_ll(x, sampled_data), x0=[0.5], bounds = [(0.001, 0.999)])
    bootstrap_results[i] = result.x
print '95% Confidence interval for p using quantiling:', np.percentile(bootstrap_results, [2.5, 97.5])

95% Confidence interval for p using quantiling: [ 0.155351    0.93221912]


***

5. Linear Regression - Extra Credit (7 EC Points)
===

Repeat problem 3 from HW 10 using MLEs. The slope only model is a restricted version of the slope/intercept model. Using a likelihood ratio test, determine if the intercept is necessary (provide a $p$-value). I've copied the data below. You do not need to repeat the normality test nor the correlation test.

In [131]:
temperature = np.array([297.70, 309.02,  327.41,  352.27,  370.60,  376.25,  406.70,  420.97,  433.88,  448.29])
pressure = np.array([57.07, 64.87,  68.93,  74.13,  77.40,  74.83,  81.22,  85.72,  90.35,  89.99])